## 比较检验

看起来似乎有了获取测试集(这里我们没有区分验证，测试集！！)的评估方法和用于比较模型的性能度量之后，就能够通过不同模型在测试集(验证集-这里姑且都叫测试集)上的性能表现来判断优劣了。但是！事实上，在机器学习中，模型比较并不是这样简单的比大小，而是要考虑更多。

在模型比较中，主要有以下三个重要考虑：

1. 测试集上的性能只是泛化性能的近似，未必相同；
2. 测试集的选择对测试性能有很大影响，即使规模一致，但测试样例不同，结果也不同；
3. 一些机器学习算法有随机性，即便算法参数相同，在同一测试集上跑多次，结果也可能不同；

那么应该如何有效地进行模型比较呢？答案是采用**假设检验（hypothesis test）**。基于假设检验的结果，我们可以推断出，若在测试集上观察到模型A优于B，则是否A的泛化性能在统计意义上也优于B，以及做这个结论的把握有多大。

典型的有最基本的二项检验和t检验，当然还有其他检验方法。默认以错误率作为性能度量。

几个基础概念：

- **置信度**：表示有多大的把握认为假设是正确的。
- **显著度**：也称“显著性水平”，表示假设出错的概率。显著度越大，假设被拒绝的可能性越大。
- **自由度**：不被限制的样本数，也可以理解为能自由取值的样本数，记为 $v$ 或 $df$。

### 单个模型、单个数据集上的泛化性能检验

> 我们有多大把握相信对一个模型泛化性能的假设？

#### 二项检验

在进行比较检验前，完成了一次模型预测，已知测试错误率为 $\hat{\epsilon}$。

一个泛化错误率为 $\epsilon$ 的模型在 $m$ 个样本上预测错 $m'$ 个样本的概率为：

$$ P(\hat{\epsilon};\epsilon) = \binom{m}{m'} \epsilon^{m'} (1-\epsilon)^{m - m'}$$

这个概率符合二项分布:

![二项分布](http://zy.swust.net.cn/07/1/sytjff/image/3.7a.gif)

又因为已知测试错误率为 $\hat{\epsilon}$，也即知道了该模型在 $m$ 个样本上实际预测错 了$\hat{\epsilon} \times m$ 个样本。代入公式，对 $\epsilon$ 求偏导会发现，给定这些条件时，**$\epsilon = \hat{\epsilon}$ 的概率是最大的**。

使用**二项检验（binomial test）**，假设泛化错误率 $\epsilon \leq \epsilon_0$，并且设定置信度为 $1-\alpha$。则可以这样定义错误率的阈值 $\overline{\epsilon}$：

$$\overline{\epsilon} = \max{\epsilon} \qquad s.t. \qquad \sum_{i=\epsilon_0 \times m+1}^m \binom{m}{i}\epsilon^i (1-\epsilon)^{m-i} < \alpha$$

其中 $s.t.$ 表示左式在右边条件满足时成立。右式计算的是发生不符合假设的事件的总概率，如果我们要有 $1-\alpha$ 的把握认为假设成立，那么发生不符合假设的事件的总概率就必须低过 $\alpha$。

在满足右式的所有 $\epsilon$ 中，选择最大的作为阈值 $\overline{\epsilon}$。如果在测试集中观测到的测试错误率 $\hat{\epsilon}$ 是小于阈值 $\overline{\epsilon}$的， 我们就能以$1-\alpha$ 的把握认为假设成立，即该模型的泛化误差 $\epsilon \leq \epsilon_0$。

#### t检验

二项检验只用于检验某一次测试的性能度量，但实际任务中我们会进行多次的训练/测试，得到多个测试错误率，比方说进行了k次测试，得到 $\hat{\epsilon}_1$,$\hat{\epsilon}_2$, ... ,$\hat{\epsilon}_k$。这次就会用到**t检验(t-test)**。

定义这 $k$ 次测试的平均错误率 $\mu$ 和方差 $\sigma^2$：

$$\mu = \frac{1}{k} \sum_{i=1}^k \hat{\epsilon_i}$$

$$\sigma^2 = \frac{1}{k-1} \sum_{i=1}^k (\hat{\epsilon_i} - \mu)^2$$

注意！这里使用的是**无偏估计**的**样本方差**，分母是 $k-1$，因为当均值确定，并且已知 $k-1$ 个样本的值时，第 $k$ 个样本的值是可以算出来的，也可以说是**受限的**。

假设泛化错误率 $\epsilon = \epsilon_0$，并且设定显著度为 $\alpha$。计算统计量t：

$$t = \frac{\sqrt{k}(\mu-\epsilon_0)}{\sigma}$$

该统计量服从自由度 $v = k-1$ 的t分布，如下图：

![t分布](https://staff.aist.go.jp/t.ihara/t.files/image076.jpg)

自由度越大，约接近于正态分布，自由度为无穷大时变为标准正态分布（$\mu=0$，$\sigma=1$）。

如果计算出的t统计量落在临界值范围 [$t_{-a/2}$,$t_{a/2}$] 之内（注：临界值由自由度 $k$ 和显著度 $\alpha$ 决定，通过查表得出），我们就能以$1-\alpha$ 的把握认为假设成立，即该模型的泛化误差 $\epsilon = \epsilon_0$。

### 两个模型/算法、单个数据集上的泛化性能检验

> 我们有多大把握相信两个模型的泛化性能无显著差别？

#### 交叉验证t检验

对两个模型A和B，各使用k折交叉验证分别得到k个测试错误率，即$\hat{\epsilon}_1^A$,$\hat{\epsilon}_2^A$, ... ,$\hat{\epsilon}_k^A$ 和 $\hat{\epsilon}_1^B$,$\hat{\epsilon}_2^B$, ... ,$\hat{\epsilon}_k^B$。使用**k折交叉验证成对t检验（paired t-tests）**来进行比较检验。

对于这两组k个测试错误率，计算两组之间的每一对的差，即 $\triangle_i = \hat{\epsilon}_k^A - \hat{\epsilon}_k^B$，从而得到k个 $\triangle$。我们可以计算 $\triangle$ 的均值 $\mu$ 和方差 $\sigma^2$，定义统计量t：

$$t = \lvert \frac{\sqrt{k}\mu}{\sigma} \rvert$$

可以看到，和前面的t检验相比，这里的分子没有被减项，其实是省略了。因为我们假设两个模型的泛化错误率相同，实际上是假设 $\lvert \epsilon^A - \epsilon^B \rvert = 0$，这个 $0$ 被省略了。

类似地，这个统计量服从自由度 $v = k-1$ 的t分布。我们设定好显著度 $\alpha$，查表获取临界值范围，如果计算出的t统计量落在在范围内，就能以$1-\alpha$ 的把握认为假设成立，即两个模型的泛化性能无显著差别，否则认为平均测试错误率较低的模型更胜一筹。

#### McNemar检验

对于一个二分类问题，如果使用留出法，我们不仅可以获得两个算法A和B各自的测试错误率，或能够获得它们分类结果的差别（都预测正确、都预测错误、一个预测正确一个预测错误），构成一张**列联表（contingency table）**：

<table>

<tr>
    <th rowspan="2" align="center">算法B</th>
    <th colspan="2" align="center">算法A</th>
</tr>

<tr>
    <td align="center">分类正确</td>
    <td align="center">分类错误</td>
</tr>
<tr>
    <td align="center">分类正确</td>
    <td align="center">$e_{00}$</td>
    <td align="center">$e_{01}$</td>
</tr>
<tr>
    <td align="center">分类错误</td>
    <td align="center">$e_{10}$</td>
    <td align="center">$e_{11}$</td>
</tr>
</table>

假设两个算法的泛化性能无显著区别，则 $e_{01}$ 应该等于 $e_{10}$，变量 $\lvert e_{01}-e_{10} \rvert$ 应服从均值为 $1$，方差为 $e_{01} + e_{10}$ 的正态分布，可以计算统计量 $\chi^2$：

$$\chi^2 = \frac{(\lvert e_{01}-e_{10} \rvert -1)^2}{e_{01} + e_{10}}$$

该变量服从自由度为 $v=1$ 的 $\chi^2$ 分布（卡方分布），类似t检验，设定好显著度 $\alpha$，按照自由度和显著度查表获得临界值。若计算所得的统计量 $\chi^2$ 小于临界值，则能以$1-\alpha$ 的把握认为假设成立，即两个算法的泛化性能无显著差别，否则认为平均测试错误率较低的算法更胜一筹。

注：这里 $v$ 为1是因为只有2个算法

### 多个模型/算法、多个数据集上的泛化性能检验

> 我们有多大把握相信多个模型的泛化性能皆无显著差别？若有，接下来怎样做？

在**一组数据集**上进行**多个算法**的比较，情况就变得较复杂了，一种做法是使用前面的方法分开两两比较；另一种更直接的做法是使用基于算法排序的Friedman检验。

#### Friedman检验

假设有 $N=4$ 个数据集，$k=3$ 种算法，可以使用一种评估方法，获得各个算法在各个数据集上的测试结果，然后按照性能度量由好到坏进行排序，序值为1，2，3。若并列，则取序值的平均值。然后对各个算法在各数据集上的序值求平均得到平均序值，如：

| 数据集 | 算法A | 算法B | 算法C |
|:-:|:-:|:-:|:-:|
|D1|1|2|3|
|D2|1|2.5|2.5|
|D3|1|2|3|
|D4|1|2|3|
|平均序值|1|2.125|2.875|

令 $r_i$ 表示第 $i$ 个算法的平均序值，则 $r_i$ 服从均值为 $\frac{k+1}{2}$，方差为 $\frac{(k^2)-1}{12}$ 的正态分布。可以计算统计量 $\chi^2$：

$$\chi^2 = \frac{12N}{k(k+1)}(\sum_{i=1}^k r_i^2 - \frac{k(k+1)^2}{4})$$

在 $k$ 和 $N$ 都较大时(通常要求 $k>30$)，该变量服从自由度为 $v=k-1$ 的 $\chi^2$ 分布（卡方分布）。

以上这种检验方式也称为**原始Friedman检验**，被认为过于保守，现在通常用统计量 $F$ 代替：

$$F = \frac{(N-1)\chi^2}{N(k-1)-\chi^2}$$

该变量服从于自由度为 $v=k-1$ 或 $v=(k-1)(N-1)$ 的 $F$ 分布。

和前面的检验方式有所区别，F检验是根据设定的显著度 $\alpha$ 和**算法个数** $k$ 以及 **数据集个数**$N$ 这三者来查表的，如果计算出的统计量 $F$ 小于查表所得的临界值，则假设成立，能以$1-\alpha$ 的把握认为认为这 $k$ 个算法的泛化性能无显著区别。

但如果这个假设被拒绝了呢？这时就需要进行**后续检验（post-hoc test）**，常用的有 **Nemenyi后续检验**。

#### Nemenyi后续检验

定义**平均序值差别的临界值域**为：

$$CD = q_\alpha \sqrt{\frac{k(k+1)}{6N}}$$

其中 $q_\alpha$是由 显著度 $\alpha$ 和**算法个数** $k$ 确定的，通过查表获取。若两个算法的平均序值之差不超过 $CD$，则能以$1-\alpha$ 的把握认为这两个算法的泛化性能无显著区别，否则认为平均序值较小的更胜一筹。

Nemenyi后续检验还可以通过Friedman检验图更直观地体现出来，横轴为性能度量，纵轴为算法，每个算法用一段水平线段表示，线段中心点为该算法的平均序值，线段长度为 $CD$。若两个算法的线段投影到x轴上有重叠部分，则可以认为这两个算法的泛化性能无显著区别。

## 偏差与方差

除了估计算法的泛化性能，我们往往还希望知道**为什么有这样的性能**？这时一个有用的工具就是**偏差-方差分解（bias-variance decomposition）**。

知乎上面有两个问题都有不错的答案，不妨先看看。[1] [机器学习中的Bias(偏差)，Error(误差)，和Variance(方差)有什么区别和联系？](https://www.zhihu.com/question/27068705)；[2] [偏差和方差有什么区别？](https://www.zhihu.com/question/20448464)。

对学习算法的期望繁华错误率进行拆解，最终会发现能拆解为三个项（需要推导）：

$$E(f;D) = \mathbb{E}_D[(f(x;D) - \overline{f}(x))^2] + (\overline{f}(x) - y)^2 + \mathbb{E}_D[(y_D - y)^2]$$

依次对应于**方差（variance）**、**偏差（bias）**、**噪声（noise）**：

$$E(f;D) = var(x) + bias^2(x) + \epsilon^2$$

这三者的含义是这样的：

- 方差：使用同规模的不同训练集进行训练时带来的性能变化，刻画**数据扰动带来的影响**；

- 偏差：学习算法的期望预测与真实结果的偏离程度，刻画**算法本身的拟合能力**；

- 噪声：当前任务上任何算法所能达到的期望泛化误差的下界（即不可能有算法取得更小的误差），刻画**问题本身的难度**；

也即是说，泛化性能是有学习算法的拟合能力，数据的充分性以及问题本身的难度共同决定的。给定一个任务，噪声是固定的，我们需要做得就是尽量降低偏差和方差。

但是这两者其实是有冲突的，这称为**偏差-方差窘境（bias-variance dilemma）**。给定一个任务，我们可以控制算法的训练程度（如决策树的层数）。在训练程度较低时，拟合能力较差，因此训练数据的扰动不会让性能有显著变化，此时偏差主导泛化错误率；在训练程度较高时，拟合能力很强，以至于训练数据自身的一些特性都会被拟合，从而产生过拟合问题，训练数据的轻微扰动都会令模型产生很大的变化，此时方差主导泛化错误率。

**注意**，将泛化性能完美地分解为方差、偏差、噪声这三项仅在**基于均方误差的回归任务**中得以推导出，分类任务由于损失函数的跳变性导致难以从理论上推导出分解形式，但已经有很多方法可以通过实验进行估计了。

Bias（偏差）和Variance（方差）

偏差Bias：描述的是预测值（估计值）的期望与真实值之间的偏离程度。偏差越大，越偏离真实数据，如下图第二行所示。

方差Variance：描述的是预测值的变化范围，离散程度，也就是离其期望值的距离。方差越大，数据的分布越分散,如下图右列所示。

![](./img/bias_var.jpg)

偏差和方差对误差Error的影响

首先模型的泛化误差可以表示为

Error = Bias + Variance + Noise

其中，Noise（噪声）是数据本身的问题，比如不准确的客户资料，在此不进行讨论。

偏差（bias）造成的误差 - 准确率和欠拟合

如果模型具有足够的数据，但因不够复杂而无法捕捉基本关系，即如果模型不适当，则会出现偏差。这样一来，模型一直会系统地错误表示数据，从而导致预测准确率降低。出现欠拟合（underfitting）。

方差（variance）造成的误差 - 精准度和过拟合

方差就是指模型过于贴近训练数据，以至于没办法把它的结果泛化（generalize）。而泛化是正事机器学习要解决的问题，如果一个模型只能对一组特定的数据有效，换了数据就无效了，我们就说这个模型过拟合。

用一张图表示为

![](./img/model_complexity.jpg)

learning curve

![](./img/learning_curve.jpg)

很明显图右上角的模型效果最好，训练集和测试集都有很好的分数，并且收敛在一起。而右下的模型过拟合了
